---

In [10]:
#!pip install --user python-swiftclient python-keystoneclient --upgrade
#!apt-get install swiftclient
#!pip install geopandas

#Solve geopandas rtee problemm for sjoin
#!apt-get install -y libspatialindex-dev
#!pip install rtree

#!pip install nbdev

# If the graph do not display, try :
#!jupyter labextension install jupyterlab-plotly
# You may have to upgrade Node and Jupyter

# OCO2 - Display map and capture zone around the peak
Project for **Data For Good**, season 7. 

---

In [11]:
# default_exp oco2mapfolium

In [12]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Introduction


In [16]:
# export
import pandas as pd
import geopandas as gpd
import numpy as np
from numpy import exp, loadtxt, pi, sqrt, log
import math
import matplotlib
import matplotlib.pyplot as plt
import swiftclient
import json
from io import StringIO
import folium
from folium import plugins
import geopy
from shapely.geometry import Polygon
from shapely.wkt import loads
from geopy.distance import VincentyDistance

In [17]:
config_path = "../config.json"

with open(config_path) as json_data_file:
    config = json.load(json_data_file)
    
def swift_con(config):
    user=config['swift_storage']['user']
    key=config['swift_storage']['key']
    auth_url=config['swift_storage']['auth_url']
    tenant_name=config['swift_storage']['tenant_name']
    auth_version=config['swift_storage']['auth_version']
    options = config['swift_storage']['options']
    return swiftclient.Connection(user=user,
                                  key=key,
                                  authurl=auth_url,
                                  os_options=options,
                                  tenant_name=tenant_name,
                                  auth_version=auth_version)

conn = swift_con(config)

## Retrieve Data

### Gaussian peaks with captured inventory

The CSV contains the gaussian peaks unfiltered, spatially joined with inventory data. The CSV is stored on the GitHub.

In [18]:
path_peaks = "../dataset/peaks_and_invent/peaks_and_invent_1807.csv"
df = pd.read_csv(path_peaks, sep=",")
df = gpd.GeoDataFrame(df)
df['geometry'] = df['geometry'].apply(loads)
df.crs = {'init': 'epsg:4326'}
df.head()

,Unnamed: 0,sounding_id,latitude,longitude,orbit,slope,intercept,amplitude,sigma,delta,...,land_water_indicator,land_fraction,date,geometry,index_cities,number_cities,index_plants,number_plants,index_coal,number_coals
0,0,2018070110054935,39.985409,43.971569,21263,0.004107,408.164258,60.620186,16.394072,1.475165,...,0.0,100.0,2018-07-01 10:05:49.350,"POLYGON ((44.15502 40.03963, 44.03539 40.12835...",[nan],0,[nan],0,[nan],0
1,1,2018070110055077,40.039230,43.953659,21263,0.004167,408.205145,44.171865,8.901821,1.979598,...,0.0,100.0,2018-07-01 10:05:50.770,"POLYGON ((44.15255 40.10696, 44.01385 40.19985...",[nan],0,[nan],0,[nan],0
2,2,2018070110055105,40.077389,43.938644,21263,0.002886,408.176701,39.563523,9.521215,1.657726,...,0.0,100.0,2018-07-01 10:05:51.050,"POLYGON ((44.14441 40.16245, 43.98586 40.25311...",[nan],0,[nan],0,[nan],0
3,3,2018070116404004,37.705978,-54.130592,21267,-0.002662,406.739190,9.555325,2.674492,1.425326,...,1.0,0.0,2018-07-01 16:40:40.040,"POLYGON ((-54.45366 38.52462, -55.13019 38.033...",[nan],0,[nan],0,[nan],0
4,4,2018070119554403,28.719200,-100.832626,21269,-0.001500,407.612314,30.162665,10.197638,1.179995,...,0.0,100.0,2018-07-01 19:55:44.030,"POLYGON ((-101.17927 29.17243, -101.44907 28.7...",[nan],0,"[4184.0, 4189.0, 4214.0, 7025.0]",4,"[9263.0, 9264.0, 9265.0, 9266.0, 9267.0, 9268....",9


### Cities estimates

The CSV contains data from the worlds biggest cities. The CSV is stored on the GitHub.

In [19]:
path_cities = "https://raw.githubusercontent.com/dataforgoodfr/batch7_satellite_ges/master/dataset/cities_v1.csv"
cities = pd.read_csv(path_cities, sep=",", index_col=0)
cities = gpd.GeoDataFrame(cities, geometry=gpd.points_from_xy(cities.longitude, cities.latitude))
cities.crs = {'init': 'epsg:4326'}
cities.head()

,City name,Country,Scope-1 GHG emissions [tCO2 or tCO2-eq],Scope-1 source dataset,Scope-1 GHG emissions units,Year of emission,City location (CDP) [degrees],Population (CDP),Population year (CDP),latitude,longitude,geometry
0,Toronto,Canada,16151019.0,CDP,tCO2,2013,"43.653226,-79.3831843",2753100.0,2011.0,43.653226,-79.383184,POINT (-79.38318 43.65323)
1,Santiago de Cali,Colombia,NaN,CDP,tCO2-eq,2010,"3.451647,-76.531985",2369829.0,2015.0,3.451647,-76.531985,POINT (-76.53198 3.45165)
4,Milano,Italy,3728678.0,CDP,tCO2,2013,"45.802578,9.086356",1350680.0,2014.0,45.802578,9.086356,POINT (9.08636 45.80258)
5,Hayward,USA,861854.0,CDP,tCO2-eq,2015,"37.6689,-122.0808",158985.0,2015.0,37.668900,-122.080800,POINT (-122.08080 37.66890)
6,Tokyo,Japan,27611000.0,CDP,tCO2-eq,2014,"35.6896342,139.6921007",13513734.0,2015.0,35.689634,139.692101,POINT (139.69210 35.68963)


### Power Plants sources

The CSV contains data from the worlds biggest power plants. The CSV is stored on the GitHub.

In [20]:
path_plants = "https://raw.githubusercontent.com/dataforgoodfr/batch7_satellite_ges/master/dataset/CO2_emissions_centrale.csv"
plants = pd.read_csv(path_plants, sep=",", index_col=0)
plants = gpd.GeoDataFrame(plants, geometry=gpd.points_from_xy(plants.longitude, plants.latitude))
plants.crs = {'init': 'epsg:4326'}
plants.describe()

,capacity_mw,latitude,longitude,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,estimated_generation_gwh,gCO2/KWh,generation_gwh_2017_with_estimated_data,tCO2_emitted_in_2013,tCO2_emitted_in_2014,tCO2_emitted_in_2015,tCO2_emitted_in_2016,tCO2_emitted_in_2017,tCO2_emitted_in_2017_with estimated_data
count,8602.000000,8602.000000,8602.000000,3078.000000,3117.000000,3312.000000,3274.000000,3210.000000,5403.000000,8602.000000,8440.000000,3.078000e+03,3.117000e+03,3.312000e+03,3.274000e+03,3.210000e+03,8.440000e+03
mean,434.151369,27.287221,-5.885974,1045.620597,1091.104519,1120.312133,1058.376820,1134.772571,2265.492359,700.833062,1842.772024,8.687061e+05,9.019768e+05,9.131356e+05,8.718276e+05,9.289991e+05,1.538757e+06
std,690.388169,24.641181,86.714480,2578.122419,2715.041407,2675.889462,2632.241272,2674.701678,7140.625909,256.469725,5953.860708,2.528086e+06,2.625053e+06,2.607140e+06,2.559000e+06,2.588440e+06,6.046563e+06
min,1.000000,-77.847000,-179.977700,-2.653000,-262.902000,-141.014000,-127.507000,-132.033000,0.000000,443.000000,-132.033000,-1.845416e+03,-2.045378e+05,-6.246920e+04,-5.648560e+04,-5.849062e+04,-5.849062e+04
25%,16.073750,21.434975,-84.425525,0.035250,0.032000,0.025000,0.019000,0.341514,89.494709,443.000000,18.882956,2.567400e+01,2.134832e+01,1.711600e+01,1.287562e+01,2.172565e+02,1.086195e+04
50%,132.250000,35.594300,-23.547500,18.245000,16.227000,22.749000,16.377500,36.623000,737.318085,778.000000,340.484944,8.859949e+03,8.021180e+03,1.098873e+04,8.077698e+03,1.881502e+04,2.080822e+05
75%,600.000000,42.104150,79.832125,633.405479,630.858000,762.341497,583.308501,697.294500,2782.119122,1050.000000,2240.154958,3.416080e+05,3.368590e+05,4.247199e+05,3.279560e+05,3.801258e+05,1.423369e+06
max,8865.000000,71.292000,178.835900,27586.200000,28127.000000,32157.000000,30015.000000,35116.000000,450562.692350,1050.000000,450562.692350,2.896551e+07,2.953335e+07,3.376485e+07,3.151575e+07,3.687180e+07,4.730908e+08


### Coal Plants sources

The CSV contains data from the worlds biggest coal plants. The CSV is stored on the GitHub.

In [21]:
path_plants_coal = "https://raw.githubusercontent.com/dataforgoodfr/batch7_satellite_ges/master/dataset/CO2_emissions_coal_plant.csv"
plants_coal = pd.read_csv(path_plants_coal, sep=",", index_col=0)
plants_coal = plants_coal[plants_coal['Longitude'].notna()]
plants_coal = plants_coal[plants_coal['Latitude'].notna()]
plants_coal = gpd.GeoDataFrame(plants_coal, geometry=gpd.points_from_xy(plants_coal.Longitude, plants_coal.Latitude))
plants_coal.crs = {'init': 'epsg:4326'}
plants_coal.head()

,Country,Plant,Capacity (MW),Status,Status 2016 (done by Data4Good),Status 2017 (done by Data4Good),Status 2018 (done by Data4Good),Status 2019 (done by Data4Good),Combustion technology,Coal type,Latitude,Longitude,Annual CO2 (million tonnes / annum),Annual CO2 emissions (millions of tonnes) in 2016,Annual CO2 emissions (millions of tonnes) in 2017,Annual CO2 emissions (millions of tonnes) in 2018,Annual CO2 emissions (millions of tonnes) in 2019,geometry
0,Albania,Porto Romano Power Station,800,Cancelled,Not operating,Not operating,Not operating,Not operating,Ultra-super,Bituminous,41.37114,19.4252,3.217105,NaN,NaN,NaN,NaN,POINT (19.42520 41.37114)
1,Argentina,Río Turbio power station,120,Mothballed,Not operating,Not operating,Not operating,Not operating,Subcritical,Bituminous,-51.54600,-72.2313,0.609181,NaN,NaN,NaN,NaN,POINT (-72.23130 -51.54600)
2,Argentina,Río Turbio power station,120,Shelved,Not operating,Not operating,Not operating,Not operating,Subcritical,Bituminous,-51.54600,-72.2313,0.609181,NaN,NaN,NaN,NaN,POINT (-72.23130 -51.54600)
3,Argentina,San Nicolás power station,350,Operating,Operating,Operating,Operating,Operating,Subcritical,Bituminous,-33.35490,-60.1729,1.998875,1.998875,1.998875,1.998875,1.998875,POINT (-60.17290 -33.35490)
4,Australia,Anglesea power station,160,Retired,Not operating,Not operating,Not operating,Not operating,Subcritical,Lignite,-38.38650,144.1821,1.047857,NaN,NaN,NaN,NaN,POINT (144.18210 -38.38650)


---

## Map Visualization & Statistics methods

### Inventory Map

To display only the inventory data on a folium map:

In [30]:
# export
def inventory_map_only(plants, plants_coal, cities):
    """
    Create map with inventory only
    :param plants: GeoDataFrame, Dataframe containing all registered plants.
    :param plants_coal: GeoDataFrame, Dataframe containing all registered coal plants.
    :param cities: GeoDataFrame, Dataframe containing all registered big cities.
    :return:
    """
    # Initialize Map
    inventory_map = folium.Map([43, 0], zoom_start=4)
    folium.TileLayer("CartoDB dark_matter", name="Dark mode").add_to(inventory_map)

    # Adding Power plants
    plants_group = folium.FeatureGroup(name="Plants").add_to(inventory_map)
    for index, row in plants.iterrows():
        radius = row['estimated_generation_gwh']/10000
        color="#3186CC" # blue

        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        emit = str(round(row['estimated_generation_gwh'],2))
        popup_html="""<h4>"""+tooltip+"""</h4>"""+row['country_long']+"""<p><b>Emission 2018 (est):</b> """+emit+""" GWh</p>"""
        popup=folium.Popup(popup_html, max_width=450)

        plants_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            # popup=popup,
                            # tooltip=tooltip,
                            fill=True))
        
    # Adding Coal plants
    plants_coal_group = folium.FeatureGroup(name="Coal Plants").add_to(inventory_map)
    for index, row in plants_coal.iterrows():
        radius = row['Annual CO2 emissions (millions of tonnes) in 2018']/10000
        color="#FF3333" # red

        tooltip =  "["+str(round(row['Latitude'],2))+" ; "+str(round(row['Longitude'],2))+"]"
        emit = str(round(row['Annual CO2 emissions (millions of tonnes) in 2018'],2))
        popup_html="""<h4>"""+tooltip+"""</h4>"""+str(row['Plant'])+"""<p><b>Emission 2018 (est):</b> """+emit+""" GWh</p>"""
        popup=folium.Popup(popup_html, max_width=450)

        plants_coal_group.add_child(folium.CircleMarker(location=(row["Latitude"],
                                      row["Longitude"]),
                            radius=radius,
                            color=color,
                            # popup=popup,
                            # tooltip=tooltip,
                            fill=True))



    # Adding Cities
    cities_group = folium.FeatureGroup(name="Cities").add_to(inventory_map)
    for index, row in cities.iterrows():
        radius = row['Population (CDP)']/2000000
        color="#FEF65B" # yellow

        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        pop = str(round(row['Population (CDP)'],0))
        title = "" + str(row['City name']) + ", " + str(row['Country'])
        popup_html="""<h4><b>"""+row["City name"]+"""</b>, """+row["Country"]+"""</h4>"""+"""<p>"""+tooltip+"""</p>"""+"""<p>Population 2017: """+pop+"""</p>"""
        popup_html = """<h4>"""+title+"""</h4><p>"""+tooltip+"""</p>"""+"""<p><b>Population 2017:</b> """+pop+"""</p>"""
        popup=folium.Popup(popup_html, max_width=450)

        cities_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            # popup=popup,
                            # tooltip=tooltip,
                            fill=True))


    folium.map.LayerControl(collapsed=False).add_to(inventory_map)

    plugins.Fullscreen(
        position='topright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(inventory_map)

    minimap = plugins.MiniMap()
    inventory_map.add_child(minimap)

    #inventory_map.save("inventory_map.html")
    return inventory_map



def peaks_capture_map(peaks, plants, plants_coal, cities):
    """
    Create map with peaks (marker + capture zone) and inventory
    :param peaks: GeoDataFrame, Dataframe containing the peaks we want to display.
    :param plants: GeoDataFrame, Dataframe containing all registered plants.
    :param plants_coal: GeoDataFrame, Dataframe containing all registered coal plants.
    :param cities: GeoDataFrame, Dataframe containing all registered big cities.
    :return:
    """
    # Initialize Map
    peaks_capture = folium.Map([40, -100], zoom_start=4)
    folium.TileLayer("CartoDB dark_matter", name="Dark mode").add_to(peaks_capture)

    # Adding Power plants
    plants_group = folium.FeatureGroup(name="Plants").add_to(peaks_capture)
    for index, row in plants.iterrows():
        color="#999900" 
        radius = row['estimated_generation_gwh']/10000

        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        emit = str(round(row['estimated_generation_gwh'],2))
        popup_html="""<h4>"""+tooltip+"""</h4>"""+row['country_long']+"""<p><b>Emission 2018 (est):</b> """+emit+""" GWh</p>"""
        popup=folium.Popup(popup_html, max_width=450)
        
        plants_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=0.1,
                            color=color,
                            tooltip=tooltip,
                            popup=popup,
                            fill=True))
        
    # Adding Coal plants
    plants_coal_group = folium.FeatureGroup(name="Coal Plants").add_to(peaks_capture)
    for index, row in plants_coal.iterrows():
        color="#FF3333" # red
        radius = row['Annual CO2 emissions (millions of tonnes) in 2018']/10000
        
        tooltip =  "["+str(round(row['Latitude'],2))+" ; "+str(round(row['Longitude'],2))+"]"
        emit = str(round(row['Annual CO2 emissions (millions of tonnes) in 2018'],2))
        popup_html="""<h4>"""+tooltip+"""</h4>"""+str(row['Plant'])+"""<p><b>Emission 2018 (est):</b> """+emit+""" GWh</p>"""
        popup=folium.Popup(popup_html, max_width=450)
        
        plants_coal_group.add_child(folium.CircleMarker(location=(row["Latitude"],
                                      row["Longitude"]),
                            radius=radius,
                            color=color,
                            tooltip=tooltip,
                            popup=popup,
                            fill=True))

        
    # Adding Cities 
    cities_group = folium.FeatureGroup(name="Cities").add_to(peaks_capture)
    for index, row in cities.iterrows():
        color="#990099" 
        radius = row['Population (CDP)']/2000000
        
        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        pop = str(round(row['Population (CDP)'],0))
        title = "" + str(row['City name']) + ", " + str(row['Country'])
        popup_html="""<h4><b>"""+row["City name"]+"""</b>, """+row["Country"]+"""</h4>"""+"""<p>"""+tooltip+"""</p>"""+"""<p>Population 2017: """+pop+"""</p>"""
        popup_html = """<h4>"""+title+"""</h4><p>"""+tooltip+"""</p>"""+"""<p><b>Population 2017:</b> """+pop+"""</p>"""
        popup=folium.Popup(popup_html, max_width=450)
        
        cities_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip=tooltip,
                            popup=popup,
                            fill=True))

    # Adding detected peaks
    peaks_group = folium.FeatureGroup(name="Peaks").add_to(peaks_capture)
    peaks_group_capture = folium.FeatureGroup(name=" - 50km CirclesCapture Zone").add_to(peaks_capture)
    for index, row in peaks.iterrows():
        radius = row["amplitude"]/20
        tooltip =  "["+str(round(row['latitude'],2))+" ; "+str(round(row['longitude'],2))+"]"
        color="#FF3333" # red
        sounding = str(row['sounding_id'])
        date = str(row['date'])
        orbit = str(row['orbit'])  
        
        popup_html="""<h4>"""+tooltip+"""</h4>"""+date+"""<p>sounding_id: """+sounding+"""</br>orbit: """+orbit+"""</p>"""
        popup_html+='<p><input type="button" value="Show plot"'
        # Injecting JavaScript in popup to fire the Dash Callback
        popup_html+='onclick="\
            let bco_input = parent.document.getElementById(\'input_sounding\'); \
            let lastValue = bco_input.value;'
        popup_html+=f'bco_input.value = \'{sounding}\';'
        popup_html+="let bco_event = new Event('input', { bubbles: true });\
            bco_event.simulated = true;\
            let tracker = bco_input._valueTracker;\
            if (tracker) {\
            tracker.setValue(lastValue);\
            }\
            bco_input.dispatchEvent(bco_event);\
            elt.dispatchEvent(new Event('change'));\
            \"/></p>"
        
        popup=folium.Popup(popup_html, max_width=450)
        
        peaks_group_capture.add_child(folium.GeoJson(row['geometry'], name=" - Capture Zone"))
        
        peaks_group.add_child(folium.CircleMarker(location=(row["latitude"],
                                      row["longitude"]),
                            radius=radius,
                            color=color,
                            tooltip=sounding,
                            popup=popup,
                            fill=True))

    folium.map.LayerControl(collapsed=False).add_to(peaks_capture)
    
    plugins.Fullscreen(
        position='topright',
        title='Expand me',
        title_cancel='Exit me',
        force_separate_button=True
    ).add_to(inventory_map)

    minimap = plugins.MiniMap()
    inventory_map.add_child(minimap)
    
    #peaks_capture.save("peaks_capture_map.html")
    return peaks_capture


def stats_invent(data, title="INVENTORY STATISTICS"):
    """
    Print statistics on peaks capture
    :param data: GeoDataFrame, Dataframe containing the peaks we want statistics on.
    :param title: str, the title on the printed stats.
    :return:
    """
    df = data[['number_cities', 'number_plants', 'number_coals']]

    print(" --- ", title, " --- ")
    print("Total count: ", df.number_cities.count())
    print("Cities mean and std: ", round(df.number_cities.mean(),2), " x ", round(df.number_cities.std(),2))
    print("Cities non-null count: ", df.number_cities[df.number_cities > 0].count(), " (", round(df.number_cities[df.number_cities > 0].count()/df.number_cities.count()*100,2), "%)")
    print("Plants mean and std: ", round(df.number_plants.mean(),2), " x ", round(df.number_plants.std(),2))
    print("Plants non-null count: ", df.number_plants[df.number_plants > 0].count(), " (", round(df.number_plants[df.number_plants > 0].count()/df.number_plants.count()*100,2), "%)")
    print("Coal Plants mean and std: ", round(df.number_coals.mean(),2), " x ", round(df.number_coals.std(),2))
    print("Coal Plants non-null count: ", df.number_coals[df.number_coals > 0].count(), " (", round(df.number_coals[df.number_coals > 0].count()/df.number_coals.count()*100,2), "%)")

    emitter = (df.number_cities + df.number_plants + df.number_coals) > 0
    print("Peaks without emitter: ", df.number_cities.count() - emitter.sum(), " (", round((df.number_cities.count()-emitter.sum())/df.number_coals.count()*100,2), "%)")
    emitters = (df.number_cities + df.number_plants + df.number_coals) > 5
    print("Peaks less than 5 emitters: ", df.number_cities.count() - emitters.sum(), " (", round((df.number_cities.count()-emitters.sum())/df.number_coals.count()*100,2), "%)")

## Examples and Sanity Checks

### Map Examples

In [ ]:
inventory_map_only(plants, plants_coal, cities)

In [ ]:
peaks_capture_map(df[110:115], plants, plants_coal, cities)

### Statistics example

In [28]:
stats_invent(df, "BASE DETECTION WITHOUT FILTER (Benoit)")

 ---  BASE DETECTION WITHOUT FILTER (Benoit)  --- 
Total count:  192
Cities mean and std:  0.18  x  0.61
Cities non-null count:  18  ( 9.38 %)
Plants mean and std:  13.31  x  29.05
Plants non-null count:  128  ( 66.67 %)
Coal Plants mean and std:  18.91  x  53.78
Coal Plants non-null count:  100  ( 52.08 %)
Peaks without emitter:  62  ( 32.29 %)
Peaks less than 5 emitters:  87  ( 45.31 %)


In [31]:
from nbdev.export import *
notebook2script()

Converted 03_25_OCO2_Data_Exploration.ipynb.
Converted 04_01_OCO2_Work_Base.ipynb.
Converted 04_04_OCO2_China_Peaks.ipynb.
Converted 04_15_OCO2_Laiwu_Peak_Detection.ipynb.
Converted CO2_emissions_Inventory_data.ipynb.
Converted Find_Peaks_with_LSTM_autoencoders.ipynb.
Converted index.ipynb.
Converted Laiwu_Plume-more_data.ipynb.
Converted Laiwu_Plume-more_data_CD_exploration_selection_peaks.ipynb.
Converted Laiwu_Plume.ipynb.
Converted oco2peak-datasets.ipynb.
Converted oco2peak-find_peak.ipynb.
Converted oco2peak-map.ipynb.
Converted oco2peak-nc4_convert.ipynb.
Converted oco2peak-swift_utils.ipynb.
Converted oco2peak_find_source.ipynb.
Converted show_map.ipynb.
Converted view_peak.ipynb.
Converted WIP_OCO2_Capture.ipynb.
Converted WIP_OCO2_Data_Capture.ipynb.
Converted WIP_OCO2_Map.ipynb.
Converted WIP_OCO2_Peaks_Wind.ipynb.
Converted WIP_OCO2_Peaks_Wind_Visualization.ipynb.
